<a href="https://colab.research.google.com/github/nirmit27/ds-and-ml/blob/main/DL/PyTorch/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Network**
Designing a neural network by leveraging the `torch.nn` module.

### Importing the necessary libraries

In [ ]:
!pip install torchinfo

In [ ]:
import os
from pprint import pprint


import numpy as np
import pandas as pd


from sklearn.preprocessing import LabelEncoder as LE
from sklearn.preprocessing import StandardScaler as SS
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score as accuracy


import torch
from torch import nn, optim
from torchinfo import summary

## **Modelling**
Defining the neural network in question.

### Neuron
A single neuron that has a **sigmoid** function output.

In [ ]:
class Model_1(nn.Module):

  def __init__(self, num_features):
    super().__init__()

    self.network = nn.Sequential(
      nn.Linear(num_features, 1),
      nn.Sigmoid(),
    )

  def forward(self, features):
    out = self.network(features)
    return out

#### Instantiating the model

In [ ]:
torch.manual_seed(42)
features = torch.rand(10, 5)

model_1 = Model_1(features.shape[1])
model_1(features)

tensor([[0.4769],
        [0.4913],
        [0.4193],
        [0.4561],
        [0.5514],
        [0.5342],
        [0.4267],
        [0.4170],
        [0.5117],
        [0.5060]], grad_fn=<SigmoidBackward0>)

#### Generating the model summary

In [ ]:
summary(model_1, features.shape)

Layer (type:depth-idx)                   Output Shape              Param #
Model_1                                  [10, 1]                   --
├─Sequential: 1-1                        [10, 1]                   --
│    └─Linear: 2-1                       [10, 1]                   6
│    └─Sigmoid: 2-2                      [10, 1]                   --
Total params: 6
Trainable params: 6
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

### Neural Network
Defining a model that comprises a **hidden** layer in addition to the input and output layers.

In [ ]:
class Model_2(nn.Module):

  def __init__(self, num_features):
    super().__init__()

    self.network = nn.Sequential(
      nn.Linear(num_features, 3),
      nn.ReLU(),
      nn.Linear(3, 1),
      nn.Sigmoid(),
    )

  def forward(self, features):
    out = self.network(features)
    return out

In [ ]:
torch.manual_seed(33)

features_2 = torch.rand(10, 5, )
model_2 = Model_2(features_2.shape[1])

model_2(features_2)

tensor([[0.4743],
        [0.4830],
        [0.4860],
        [0.4657],
        [0.4696],
        [0.4780],
        [0.4673],
        [0.4768],
        [0.4635],
        [0.4628]], grad_fn=<SigmoidBackward0>)

In [ ]:
summary(model_2, features_2.shape)

Layer (type:depth-idx)                   Output Shape              Param #
Model_2                                  [10, 1]                   --
├─Sequential: 1-1                        [10, 1]                   --
│    └─Linear: 2-1                       [10, 3]                   18
│    └─ReLU: 2-2                         [10, 3]                   --
│    └─Linear: 2-3                       [10, 1]                   4
│    └─Sigmoid: 2-4                      [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

## Training
Training the neural network on the sample **Breast Cancer** dataset for **Binary Classification** task.

### Importing the dataset

In [ ]:
df = pd.read_csv(r"https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


### Preprocessing

#### Dropping unwanted columns

In [ ]:
df.drop(columns=["id", "Unnamed: 32"], inplace=True)
df.columns

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

#### Splitting the dataset

In [ ]:
test_size, random_state = 0.2, 44
X_train, X_test, y_train, y_test = tts(df.iloc[:, 1:], df.iloc[:, 0], test_size=test_size, random_state=random_state)

X_train.shape, X_test.shape

((455, 30), (114, 30))

#### Scaling the features

In [ ]:
ss = SS()
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

#### One-hot Encoding the target labels

In [ ]:
le = LE()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

y_test

array([1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1])

#### Converting `numpy` arrays to `torch` tensors

In [ ]:
X_train, X_test, y_train, y_test = torch.from_numpy(X_train), torch.from_numpy(X_test), torch.from_numpy(y_train), torch.from_numpy(y_test)
X_train.shape

torch.Size([455, 30])

In [ ]:
y_test

tensor([1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
        1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1])

## Testing
Let's put a our sample neural network to the test!

In [ ]:
class Model_3(nn.Module):

  def __init__(self, num_features):
    super().__init__()

    self.network = nn.Sequential(
        nn.Linear(num_features, 1),
        nn.Sigmoid()
    )

  def forward(self, features):
    out = self.network(features)
    return out

  def loss_function(self, y_pred, y):
    criterion = nn.BCELoss()
    loss = criterion(y_pred, y.type(y_pred.dtype).reshape(y_pred.shape))
    return loss

#### Hyperparameters

In [ ]:
epochs = 25
lr = 0.5

#### Pipeline
Using the built-in **Stochastic Gradient Descent** optimizer function and **Binary Cross-Entropy** loss functions for updating model's weights and biases and computing the loss function respectively.

In [ ]:
model = Model_3(X_train.shape[1])
optimizer = optim.SGD(model.parameters(), lr=lr)

for ep in range(epochs):
  y_pred = model(X_train.float())
  loss = model.loss_function(y_pred, y_train)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  print(f"Epoch : {ep + 1}\tLoss : {loss:.4f}")

Epoch : 1	Loss : 0.4705
Epoch : 2	Loss : 0.2634
Epoch : 3	Loss : 0.2204
Epoch : 4	Loss : 0.1951
Epoch : 5	Loss : 0.1780
Epoch : 6	Loss : 0.1656
Epoch : 7	Loss : 0.1562
Epoch : 8	Loss : 0.1487
Epoch : 9	Loss : 0.1426
Epoch : 10	Loss : 0.1376
Epoch : 11	Loss : 0.1332
Epoch : 12	Loss : 0.1295
Epoch : 13	Loss : 0.1262
Epoch : 14	Loss : 0.1233
Epoch : 15	Loss : 0.1208
Epoch : 16	Loss : 0.1184
Epoch : 17	Loss : 0.1163
Epoch : 18	Loss : 0.1144
Epoch : 19	Loss : 0.1126
Epoch : 20	Loss : 0.1110
Epoch : 21	Loss : 0.1094
Epoch : 22	Loss : 0.1080
Epoch : 23	Loss : 0.1067
Epoch : 24	Loss : 0.1055
Epoch : 25	Loss : 0.1043


#### Evaluation

In [ ]:
with torch.no_grad():
  y_hat = (model(X_test.float()) > 0.5).float()

  score = accuracy(y_test, y_hat.round())
  print(f"Accuracy score : {(score * 100):.4f}%")

Accuracy score : 98.2456%
